<a href="https://colab.research.google.com/github/aquamarine-28/Reality-Check/blob/main/model_Edited_4_6_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reality Check


An Instance Segementation model that detects instances of AI-generated features and/or manipulation in images of people (ie. deepfakes).

(Link y'alls GitHubs or LinkedIns...or we could just delete this tbh)

By: Mark-Anthony Delva,
    [Nmesoma Duru](https://github.com/aquamarine-28), and
    Micheal Johnson

In [ ]:
!pip install ultralytics
!pip install keras
!pip install roboflow

# Import necessary modules

In [ ]:
from google.colab import userdata
k1 = userdata.get('api_key')
k2 = userdata.get('api_key_Mi')

import os
import ultralytics
from ultralytics import YOLO
from IPython import display
from IPython.display import display, Image
from roboflow import Roboflow

ultralytics.checks()

Ultralytics YOLOv8.1.43 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.2/78.2 GB disk)


# Download the datasets

Real images

In [ ]:
### Datasets of annotated images from Roboflow
# There are two different datasets, so they would have to be merged

rf = Roboflow(api_key = k1)

print(rf.workspace())

# Real images download
project = rf.workspace("reality-check").project("reality_check_ai_detection-2")
version = project.version(1)
dataset = version.download("yolov8")

Fake images

In [ ]:
rf2 = Roboflow(api_key = k2)

print(rf2.workspace())

# Fake images download
project2 = rf2.workspace("reality-check-lx25q").project("fake-images-2")
version2 = project.version(1)
dataset2 = version.download("yolov8")

# Make directories for the 'Train,' 'Test,' and 'Validation' images

In [ ]:
# TODO: merge the datasets of real and fake images
# Train the model
# Test and validate:
# Work on deployment

# Train the model

# Validation and Testing